In [1]:
!pip install beautifulsoup4 pandas selenium webdriver-manager

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 10.6 MB/s  0:00:01 11.1 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 11.0 MB/s  0:00:00 11.6 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 10.9 MB/s  0:00:01a 0:00:010:00:0101
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14/14 [selenium] 13/14 [selenium]ocket]t-client]


**Setup-Teste inicial para o web scrapping**

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import random as rd
import time

url = 'https://www.moneytimes.com.br/empresas'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

response = requests.get(url, headers=headers)

print(f"Status Code: {response.status_code}")

Status Code: 200


**Scrapping para o site moneytimes.com.br**

In [5]:
i = 0
dados = []
while response.status_code == 200 and i<=3500:

  soup = BeautifulSoup(response.content, 'html.parser')

  divNoticia = soup.find_all(class_='news-item')

  count = 0

  for k, item in enumerate(divNoticia):
        try:

            titulo = item.find('h2', class_= 'news-item__title')
            data = item.find('span', class_='date')
            if titulo:
                titulo = titulo.get_text(strip=True)
                if data:
                    data = data.get_text(strip=True)
                    registro = {
                    'Titulo': titulo,
                    'Data': data
                }
            dados.append(registro)
            count+=1
        except Exception as e:
            print(f"Erro ao ler item {k}: {e}")
  print(f"Página {i}: {len(divNoticia)} elementos encontrados -> {count} coletados com sucesso. (Total acumulado: {len(dados)})")
  i+=1
  url = f'https://www.moneytimes.com.br/empresas/page/{i}/'
  headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
  response = requests.get(url, headers=headers)
  pausa = rd.uniform(1, 3)
  time.sleep(pausa)

scrapping = pd.DataFrame(dados)

scrapping.tail()


Página 0: 9 elementos encontrados -> 9 coletados com sucesso. (Total acumulado: 9)
Página 1: 9 elementos encontrados -> 9 coletados com sucesso. (Total acumulado: 18)
Página 2: 9 elementos encontrados -> 9 coletados com sucesso. (Total acumulado: 27)
Página 3: 9 elementos encontrados -> 9 coletados com sucesso. (Total acumulado: 36)
Página 4: 9 elementos encontrados -> 9 coletados com sucesso. (Total acumulado: 45)
Página 5: 9 elementos encontrados -> 9 coletados com sucesso. (Total acumulado: 54)
Página 6: 9 elementos encontrados -> 9 coletados com sucesso. (Total acumulado: 63)
Página 7: 9 elementos encontrados -> 9 coletados com sucesso. (Total acumulado: 72)
Página 8: 9 elementos encontrados -> 9 coletados com sucesso. (Total acumulado: 81)
Página 9: 9 elementos encontrados -> 9 coletados com sucesso. (Total acumulado: 90)
Página 10: 9 elementos encontrados -> 9 coletados com sucesso. (Total acumulado: 99)
Página 11: 9 elementos encontrados -> 9 coletados com sucesso. (Total acumul

,Titulo,Data
31504,Petrobras diz que controle operário em subsidi...,09 abr 2019
31505,TCU: 86% dos salários das estatais são superio...,09 abr 2019
31506,Número de empresas industriais que investem em...,09 abr 2019
31507,Trabalhadores movem ação de R$ 5 bilhões contr...,09 abr 2019
31508,Trabalhadores movem ação de R$ 5 bilhões contr...,09 abr 2019


In [ ]:
scrapping.to_csv('./dados/moneyTimes.csv', index=False, encoding='utf-8-sig')

**Limpeza dos dados e transformação pra data**

In [ ]:
scrapping = pd.read_csv('./dados/moneyTimes.csv')

scrapping = scrapping[~scrapping['Data'].str.contains('atrás', case=False, na=False)].copy()

mapeamento = {
    'jan': '01', 'fev': '02', 'mar': '03', 'abr': '04', 'mai': '05', 'jun': '06',
    'jul': '07', 'ago': '08', 'set': '09', 'out': '10', 'nov': '11', 'dez': '12'
}

def formatar(texto):
    try:
        dia, abrev, ano = texto.lower().split()
    
        mes = mapeamento[abrev]
        
        return f"{ano}-{mes}-{dia.zfill(2)}"
    except:

        return None
scrapping['data'] = scrapping['Data'].apply(formatar)

scrapping['data'] = pd.to_datetime(scrapping['data'])

print(scrapping[['Data', 'data']].head())
scrapping = scrapping.drop(columns=['Data'])
scrapping.tail()

scrapping = scrapping.sort_values(by='data', ascending=False)

            Data       data
577  24 out 2025 2025-10-24
578  24 out 2025 2025-10-24
579  24 out 2025 2025-10-24
580  24 out 2025 2025-10-24
581  24 out 2025 2025-10-24


In [17]:
scrapping.tail()
scrapping.to_csv('moneyTimesFormatado.csv', index=False, encoding='utf-8-sig')